In [ ]:
!pip install roboflow
!pip install ultralytics
import yaml
import os
from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import clear_output
from google.colab import drive

# --- ¡NUEVO PASO! Montar Google Drive ---
print("Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Drive Montado.")

# --- 1. Descargar tu NUEVO Dataset Aumentado desde Roboflow ---
print("Descargando tu nuevo dataset desde Roboflow...")
rf = Roboflow(api_key="tZR7XZhFvySrbVM3hfH4") # Tu API key
project = rf.workspace("luchini").project("datasetenproceso-irzcf")

# ----- ¡ACCIÓN REQUERIDA! -----
# (la que tiene los aumentos de Roboflow y la división 92/5/3)
version = project.version(18)
dataset = version.download("yolov8")

# --- 2. Corrección Automática del data.yaml (para evitar errores de ruta) ---
data_yaml_path = f'{dataset.location}/data.yaml'
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

data_yaml['train'] = f"{dataset.location}/train/images"
data_yaml['val'] = f"{dataset.location}/valid/images"
if os.path.exists(f"{dataset.location}/test/images"):
    data_yaml['test'] = f"{dataset.location}/test/images"

with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

clear_output()
print(f"✅ Dataset (versión {version.version}) descargado y listo.")
print(f"La ubicación del config es: {data_yaml_path}")

# --- 3. Cargar el Modelo ---
# Usamos el modelo 'medium', que dio buenos resultados
# --- Cargar Modelo ---
# Mantén el modelo que estabas usando (L o X, según tu última decisión)
# Para este ejemplo, sigo con 'L' como en tu log inicial, pero ajusta si es 'X'
# --- Cargar Modelo ---
model = YOLO('yolov8n-seg.pt') # Modelo L

# --- Iniciar Entrenamiento ---
print("\nIniciando entrenamiento (Modelo L, 1088px, Batch 16, Menos Agresivo)...")
results = model.train(
    # --- Parámetros Principales ---
    data=data_yaml_path,
    epochs=500,
    patience=40,
    batch=16,            # <-- MANTENER 16 por ahora
    imgsz=720,
    rect=True,
    optimizer='AdamW',
    # *** ¡CAMBIO CLAVE! Learning Rate reducido ***
    lr0=1e-3,       # Tasa de aprendizaje inicial (0.001)
    lrf=1e-4,       # Tasa de aprendizaje final (0.0001)

   # lr0=1e-3,           # Reducido (antes 0.01)
   # lrf=1e-3,          # Reducido (antes 0.01)

    # --- Parámetros de Guardado ---
    project='/content/drive/MyDrive/YOLOv8_Training',
    # *** ¡USA UN NOMBRE NUEVO! ***
    name='Coffee_Seg_n-Model_StableAug_720px_Batch16_v1n',
    exist_ok=True,

    # --- Aumentos "On-the-Fly" (Agresivos, PERO SIN MIXUP) ---
    degrees=15.0,
    translate=0.1,
    scale=0.5,
    shear=5.0,
    flipud=0.5,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    mosaic=1.0,
    mixup=0.0,           # <-- CAMBIO CLAVE: Desactivado
    copy_paste=0.1       # Mantenemos Copy Paste
)

print("\n✅ Entrenamiento completado.")

✅ Dataset (versión 18) descargado y listo.
La ubicación del config es: /content/DatasetEnProceso-18/data.yaml

Iniciando entrenamiento (Modelo L, 1088px, Batch 16, Menos Agresivo)...
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/DatasetEnProceso-18/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=720, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.0, mode=train